In [ ]:
from math import inf
import cv2
import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

import threading as thd

mp_drawing  = mp.solutions.drawing_utils

# old

In [ ]:
mp_face_mesh = mp.solutions.face_mesh
# left eyes landmarks
LEFT_EYE  = [ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
# right eyes landmarks
RIGHT_EYE = [ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 

LEFT_IRIS  = [474,475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

Left_eb = [107,66,105,63,70,46,53,52,65,55]


######################
#'.\\Data\\Train_video.mp4' r'.\previous-MLcode\Train_video.mp4'
cap = cv2.VideoCapture(0)
#DIMS  = np.array([460, 880], dtype=int)      #Video crop dimension to subject
#SCALE = 0.75
######################

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
) as face_mesh:
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        
        ############################################
        '''
        frame = frame[300:300+DIMS[1], 80:150+DIMS[0]]
        frame = cv.resize(frame, (0, 0), fx = SCALE, fy = SCALE)
        frame = cv.flip(frame,0)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(frame)
        '''
        ############################################
        '''
        for face_landmark in results.multi_face_landmarks: 
        
            mp_drawing.draw_landmarks(
                    image = frame,
                    landmark_list = mesh_points,
                    connections   = mp_face_mesh.FACEMESH_TESSELATION,
        
                    landmark_drawing_spec = mp_drawing.DrawingSpec(
                        color=(255,0,0),
                        thickness=0,
                        circle_radius=1),
                    
                    connection_drawing_spec = mp_drawing.DrawingSpec(
                        color=(255,255,255),
                        thickness=1,
                        circle_radius=1)    )
        '''
        if results.multi_face_landmarks:
            #print(results.multi_face_landmarks[0].landmark)
            
            mesh_points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
            # multiplying with height and width to get the actual coordinates of the landmarks from {x:0.56 y:0.21} to 200, 300 etc.
            
            # print(mesh_points.shape)
            
            
            cv2.polylines(frame, [mesh_points[LEFT_IRIS]], True, (0,255,0), 1, cv2.LINE_AA)
            cv2.polylines(frame, [mesh_points[RIGHT_IRIS]], True, (0,255,0), 1, cv2.LINE_AA)
            
            #print(results.multi_face_landmarks[0].landmark)
        cv2.imshow('img', frame)
        key = cv2.waitKey(1)
        if key ==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
a = results.multi_face_landmarks[0].landmark[0]
a.x

# Detectors

In [ ]:
# https://mediapipe.readthedocs.io/en/latest/solutions/face_detection.html

class FaceDetector:
    '''
    FaceDetector is used to get the 'bounds' for a face.
    'bounds' are used to crop the image befor eseonding it for face/iris landmark detection.
    '''
    def __init__(self):
        # creating detector object
        with open(r".\data\blaze_face_short_range.tflite", "rb") as model_file:
            model_data = model_file.read()
        
        options = vision.FaceDetectorOptions(
            base_options = python.BaseOptions(model_asset_buffer=model_data),
            running_mode = vision.RunningMode.IMAGE
        )
        
        self.face_detector = vision.FaceDetector.create_from_options(options)
        
    def detect_face_bounds(self, image:np.array) -> tuple:
        # convert nump image to mediapipe format
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        
        # detect face in image
        self.face_detection_result = self.face_detector.detect(mp_image)
    
        # if face detected, draw on image and return bounds
        if self.face_detection_result.detections:
            bbox = self.face_detection_result.detections[0].bounding_box
            self.draw_bounds(image, bbox)
            return self.expand_bounds(bbox, scale=2)      # double width and height

        print("No face detected")
        return None

    
    def draw_bounds(self, image:np.array, bbox) -> tuple:
        height, width, _ = image.shape
        start_point = bbox.origin_x, bbox.origin_y
        end_point   = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
        
        cv2.rectangle(image, start_point, end_point, (255,0,0), 3)

    
    def expand_bounds(self, bbox, scale:int=2):
        x = max(0, round( bbox.origin_x - bbox.width*(scale-1)/2 )  )
        y = max(0, round( bbox.origin_y - bbox.height*(scale-1)/2 ) )

        #print(x,y)
        bbox.x = x
        bbox.y = y
        bbox.width  = round(scale*bbox.width)
        bbox.height = round(scale*bbox.height)

        return bbox

In [ ]:
class FaceMeshDetector:
    def __init__(self):
        self.LEFT_IRIS  = list(LANDMARKS_LOC['iris_L'])
        self.RIGHT_IRIS = list(LANDMARKS_LOC['iris_R'])
        self.mp_face_mesh = mp.solutions.face_mesh
        self.face_mesh    = self.mp_face_mesh.FaceMesh(refine_landmarks=True)

    def detect(self, image:np.array):
        results = self.face_mesh.process(image)
        self.draw_face_landmarks(image, results)   # OPTIONAL
        
        return results
    
    def draw_face_landmarks(self, image, results):
        if not results.multi_face_landmarks:
            return

        img_h, img_w = image.shape[:2]
        mesh_points = np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) for p in results.multi_face_landmarks[0].landmark])
        cv2.polylines(image, [mesh_points[self.LEFT_IRIS]], True, (0,255,0), 1, cv2.LINE_AA)
        cv2.polylines(image, [mesh_points[self.RIGHT_IRIS]], True, (0,255,0), 1, cv2.LINE_AA)

        
        for face_landmark in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(
                image = image,
                landmark_list = face_landmark,
                connections   = self.mp_face_mesh.FACEMESH_TESSELATION,
    
                landmark_drawing_spec = mp_drawing.DrawingSpec(
                    color=(255,0,0),
                    thickness=0,
                    circle_radius=1),
                
                connection_drawing_spec = mp_drawing.DrawingSpec(
                    color=(255,255,255),
                    thickness=1,
                    circle_radius=1)    )

In [ ]:
class HolisticDetector:
    def __init__(self):
        self.mp_holistic = mp.solutions.holistic
        self.holistic    = self.mp_holistic.Holistic()

    def detect(self, image:np.array):
        results = self.holistic.process(image)
        self.draw_landmarks(image, results)     # OPTIONAL
        return results

    def draw_landmarks(self, image:np.array, results):
        if not results.pose_landmarks:
            return
        h,w,_ = image.shape
        
        mp_drawing.draw_landmarks(
            image = image,
            landmark_list = results.pose_landmarks,
            connections   = self.mp_holistic.POSE_CONNECTIONS,
            
            landmark_drawing_spec = mp_drawing.DrawingSpec(
                color=(0,230,255),
                thickness=2,
                circle_radius=1),
            
            connection_drawing_spec = mp_drawing.DrawingSpec(
                color=(255,255,255),
                thickness=2,
                circle_radius=1)
            )
        for id,lm in enumerate(results.pose_landmarks.landmark):
            x = int(lm.x*w)
            y = int(lm.y*h)
            #print('in',x,y)
            cv2.circle(image,(x,y),1,(255,0,255),-1)
            cv2.putText(image,str(id),(x,y -1),cv2.FONT_HERSHEY_PLAIN,4,(200,200,200),4)
        

In [ ]:
class CamVideoManager:
    def __init__(self, TRAIN_VIDEO_NO:int, camF, camS):
        self.offsetF = self.offsetS = 0
        self.camF = camF
        self.camS = camS
        
        segments       = pd.read_csv(r"./train_videos/segments.csv", index_col="Train_no")
        self.selection = segments.loc[TRAIN_VID_NO]
        offset = self.selection["offsetF"]
        if offset < 0:
            self.offsetS = abs(offset)
        else:
            self.offsetF = abs(offset)

        self.set_frame() # initialize
        
    
    def get_intervals(self):
        intervals = []
        row = self.selection[2:].to_list()
        for i in range(0, len(row), 3):
            intervals.append((row[i], row[i+1], row[i+2]))
        return(intervals)

    
    def set_frame(self, frame_no:int=0, cam:int=0):
        if cam in {0,1}:
            self.camF.set(cv2.CAP_PROP_POS_FRAMES, frame_no+self.offsetF-1)
        if cam in {0,2}:
            self.camS.set(cv2.CAP_PROP_POS_FRAMES, frame_no+self.offsetS-1)


# Accumulator

In [ ]:
'''
- eyebrow (L&R)                x2
- lips    (L&R)                x2
- face: left right top bottom  x4
- body                         x8   < change here
- iris    (L&R)                x2
= 18 columns : x and y         = 36 features
'''

columns0 = ['lip_L', 'lip_R', 'brow_L', 'brow_R', 'iris_L', 'iris_R'] + [f"face{i}" for i in range(4)]  + [f"body{i}" for i in range(4)]
columns  = list()
for col in columns0:
    columns.append(col+'x')
    columns.append(col+'y')
columns.extend(['TRUTH', 'frame'])
TRAINING_FEATURES = pd.DataFrame(columns=columns)
TESTING_FEATURES  = pd.DataFrame(columns=columns)

# https://raw.githubusercontent.com/google/mediapipe/master/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png
LANDMARKS_LOC = {
    'brow_L'     : {107, 66, 105, 63, 70, 46, 53, 52, 65, 55},
    'brow_R'     : {336, 285, 296, 295, 334, 282, 293, 283, 276, 300},
    'iris_L'     : {474, 475, 476, 477},
    'iris_R'     : {469, 470, 471, 472}, 
    'lip_L'      : {78, 191, 80, 81, 82, 95, 88, 178, 87},
    'lip_R'      : {308, 415, 324, 310, 318, 311, 402, 312, 317},
    'face0'      : {54, 68, 103, 104, 108, 69, 67, 10, 151, 338, 337, 397, 333, 332, 298, 284, 251, 301, 21, 71, 109, 297, 299},                                                                       #forehead
    'face1'      : {18, 32, 83, 140, 148, 152, 171, 175, 176, 199, 200, 201, 208, 262, 313, 369, 377, 396, 400, 421, 428},                                                       #chin
    'face2'      : {36, 50, 58, 93, 101, 111, 116, 117, 118, 123, 132, 137, 138, 147, 172, 177, 186, 187, 192, 203, 205, 206, 207, 212, 213, 214, 215, 216, 227, 228, 234},      #left_face
    'face3'      : {266, 280, 288, 323, 330, 340, 345, 346, 347, 352, 361, 366, 367, 376, 397, 401, 410, 411, 416, 423, 425, 426, 427, 432, 433, 434, 435, 436, 447, 448, 454}   #right_face
}
POSTURE_LOC = {'body0':0, 'body1':12, 'body2':14, 'body3':26} #nose, shoulder, elbow, knee
    
# video intervals where the subject answers
INTERVALS = (
    (90,  140, 1),
    (170, 220, 1),
    (250, 340, 0),
    (370, 450, 1),    # TESTING INTERVAL
    (480, 540, 1),
    (600, 660, 0),
    (inf, inf,0) )    # prevents detection on remainder video

TEST_INTERVAL = 4-1

TRAINING_FEATURES

In [ ]:
class CaptureData:
    def __init__(self, name="data"):
        self.name = name
        self.df_face = pd.DataFrame(columns=columns)
        self.df_face.set_index("frame", inplace=True)
        
        self.df_pos = pd.DataFrame(columns=columns)
        self.df_pos.set_index("frame", inplace=True)

    def __repr__(self):
        print(self.df)
        return self.name

    def record_facelandmarks(self, frame_no, results):
        nose = results.multi_face_landmarks[0].landmark[4]  #nose at index 4
        #x, y = get_cordinates(nose, DIMS)
        #cv2.circle(frame, (x, y), 2, (0,0,0), 1)
        features = dict()
        
        for feature in LANDMARKS_LOC:
            feature_loc = np.array([0,0], dtype=np.float64)
            
            for idx in LANDMARKS_LOC[feature]:
                mark = results.multi_face_landmarks[0].landmark[idx]
                feature_loc += np.array( [mark.x, mark.y] )
                
            feature_loc /= len(LANDMARKS_LOC[feature])           #average feature location
            
            result = feature_loc - np.array([nose.x, nose.y])    #normalize
            features[feature+'x'] = result[0]
            features[feature+'y'] = result[1]
        
        new_row = pd.Series(features, name = frame_no)
        self.df_face = pd.concat([self.df_face, new_row.to_frame().T])
        print(self.df_face)
        pass

    def record_pos(self, frame_no, results):
        hip = results.pose_landmarks.landmark[24]  #hip at index 24
        #x, y = get_cordinates(nose, DIMS)
        #cv2.circle(frame, (x, y), 2, (0,0,0), 1)
        features = dict()
        
        for feature in POSTURE_LOC:
            idx = POSTURE_LOC[feature]
            mark = results.pose_landmarks.landmark[idx]
            feature_loc = np.array([mark.x,mark.y], dtype=np.float64)
            
            result = feature_loc - np.array([hip.x, hip.y])    #normalize
            features[feature+'x'] = result[0]
            features[feature+'y'] = result[1]
        
        new_row = pd.Series(features, name = frame_no)
        self.df_pos = pd.concat([self.df_pos, new_row.to_frame().T])
        print(self.df_pos)
        pass

    def save_data(self):
        pass

# Main


In [ ]:
face_bounds_detect = FaceDetector()
face_mesh_detector = FaceMeshDetector()
capture_data = CaptureData()
holistic_detector  = HolisticDetector()

In [ ]:
TRAIN_VID_NO = 1
camF = cv2.VideoCapture(rf".\train_videos\train{TRAIN_VID_NO}-f.mp4")
camS = cv2.VideoCapture(rf".\train_videos\train{TRAIN_VID_NO}-s.mp4")

#print( camS.get(cv2.CAP_PROP_FRAME_COUNT) )

CAMS      = CamVideoManager(TRAIN_VID_NO, camF, camS)
INTERVALS = CAMS.get_intervals()
#print(INTERVALS)

bbox = None
while bbox is None:
    _, frame = camF.read()
    bbox = face_bounds_detect.detect_face_bounds(frame)  # keep this before while True

crop = { "x1" : bbox.y,                                  # crop.x = bbox.y, because crop is matrix
         "x2" : bbox.y+bbox.width,
         "y1" : bbox.x,
         "y2" : bbox.x+bbox.height }

CAMS.set_frame(INTERVALS[0][0])

for i in range(20):
    _, frameF = camF.read()
    _, frameS = camS.read()


    cropFx256 = frameF[crop["x1"]:crop["x2"], crop["y1"]:crop["y2"] ]
    results_face = face_mesh_detector.detect(cropFx256)
    capture_data.record_facelandmarks(i,results_face) #record face data
    cropFx256  = cv2.resize(cropFx256, (512,512))

    results_holistic = holistic_detector.detect(frameS)
    capture_data.record_pos(i,results_holistic) #record posture data
    
    cv2.imshow("front", cropFx256)
    cv2.imshow("side" , cv2.resize(frameS, (0,0), fx=0.4, fy=0.4))
    key = cv2.waitKey(1)
    if key == ord('q'):
        break


cv2.destroyAllWindows()
camF.release()
camS.release()

# MISC

In [ ]:
x.results_holistic.pose_landmarks.landmark[0]

In [ ]:
for feature in LANDMARKS_LOC:
    print(feature)

features value 1_frame:
{'brow_Lx': -0.06354176104068754, 'brow_Ly': -0.0748566150665283, 'brow_Rx': 0.07440438866615295, 'brow_Ry': -0.08823261857032777, 'iris_lx': 0.07112482190132141, 'iris_ly': -0.046568095684051514, 'iris_rx': -0.05164928734302521, 'iris_ry': -0.03550254553556442, 'lip_Lx': -0.01693820291095305, 'lip_Ly': 0.06732548938857186, 'lip_Rx': 0.04028668668535018, 'lip_Ry': 0.06273240513271755, 'face0x': 0.011293470859527588, 'face0y': -0.08929244331691577, 'face1x': 0.012932631231489689, 'face1y': 0.13295339260782513, 'face2x': -0.08397503245261406, 'face2y': 0.046176983464148724, 'face3x': 0.11203789903271588, 'face3y': 0.0337283505547431}

In [ ]:
results_face.multi_face_landmarks[0].landmark[65]

In [ ]:
import pandas as pd
features = {'A': [1, 2, 3],
            'B': ['a', 'd', 'c'],
            'C': [55, 21, 32]}
# Existing DataFrame
df = pd.DataFrame(features)

# New row to add
new_row = pd.DataFrame({'B': 4,
                        'A': ['d']})

# Concatenate the existing DataFrame and the new row
df = pd.concat([df, new_row], ignore_index=True)
print(df)

In [ ]:
new_row = pd.DataFrame({'B': 55,
                        'A': ['d']})
df = pd.concat([df, new_row], ignore_index=True)
print(df)

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
capture_data.df_face